# Get config.h log from buildbot given the build url

In [26]:
builder_url = "https://www.octopus-code.org/buildbot/#/builders/144/builds/194"
config_page = f"{builder_url}/steps/5/logs/config_h"

In [2]:
import requests

In [14]:
response = requests.get(
    'https://www.octopus-code.org/buildbot/api/v2/builders/144/builds/194',
)

In [15]:
response.content

b'{\n  "builds": [\n    {\n      "builderid": 144,\n      "buildid": 61409,\n      "buildrequestid": 64717,\n      "complete": true,\n      "complete_at": 1701857397,\n      "masterid": 1,\n      "number": 194,\n      "properties": {},\n      "results": 2,\n      "started_at": 1701856191,\n      "state_string": "failed test (failure)",\n      "workerid": 24\n    }\n  ],\n  "meta": {}\n}'

In [19]:
response = requests.get('https://www.octopus-code.org/buildbot/api/v2/builds/61409/steps/5') # buildid from above


In [20]:
response.content

b'{\n  "meta": {},\n  "steps": [\n    {\n      "buildid": 61409,\n      "complete": true,\n      "complete_at": 1701856267,\n      "hidden": false,\n      "name": "configure",\n      "number": 5,\n      "results": 0,\n      "started_at": 1701856215,\n      "state_string": "configure",\n      "stepid": 568186,\n      "urls": []\n    }\n  ]\n}'

In [21]:
response = requests.get(
    'https://www.octopus-code.org/buildbot/api/v2/steps/568186/logs/config_h',
)
# stepid from above

In [22]:
response.content

b'{\n  "logs": [\n    {\n      "complete": true,\n      "logid": 709753,\n      "name": "config.h",\n      "num_lines": 551,\n      "slug": "config_h",\n      "stepid": 568186,\n      "type": "s"\n    }\n  ],\n  "meta": {}\n}'

In [23]:
response = requests.get(
    'https://www.octopus-code.org/buildbot/api/v2/logs/709753/contents',
)
# logid from above

In [24]:
response.content

b'{\n  "logchunks": [\n    {\n      "content": "o/* config.h.  Generated from config.h.in by configure.  */\\no/* config.h.in.  Generated from configure.ac by autoheader.  */\\no\\no/* date when configure was launched */\\no#define BUILD_TIME \\"Wed Dec  6 10:51:00 CET 2023\\"\\no\\no/* C compiler */\\no#define CC \\"gcc \\"\\no\\no/* The C type of a Fortran integer */\\no#define CC_FORTRAN_INT int\\no\\no/* C compiler flags */\\no#define CFLAGS \\"-Wall -O2 -march=native -ftest-coverage -fprofile-arcs\\"\\no\\no/* C compiler flags (extra) */\\no#define CFLAGS_EXTRA \\"\\"\\no\\no/* C++ compiler */\\no#define CXX \\"g++ \\"\\no\\no/* C++ compiler flags */\\no#define CXXFLAGS \\"-Wall -O2 -march=native -ftest-coverage -fprofile-arcs\\"\\no\\no/* C++ compiler flags (extra) */\\no#define CXXFLAGS_EXTRA \\"\\"\\no\\no/* Define to 1 if using \'alloca.c\'. */\\no/* #undef C_ALLOCA */\\no\\no/* compiler supports line-number lines */\\no#define F90_ACCEPTS_LINE_NUMBERS 1\\no\\no/* Fortran comp

# MVP function

In [2]:
import json
import requests
import re

In [3]:
def get_config_h(build_url:str)->str:
    """
    Return the contents of the config.h file from the buildbot

    Assuming build_url is of the form:
    builder_url = "https://www.octopus-code.org/buildbot/#/builders/144/builds/194"
    return the contents of the config.h file
    by first finding the buildid then the stepid then the logid
    """
    # get builders_id and builds_id
    # match for the pattern: builders/144 from that get the 144 for builders_id
    builders_id = re.search(r'builders/(\d+)', build_url).group(1)
    # match for the pattern: builds/194 from that get the 194 for builds_id
    builds_id = re.search(r'builds/(\d+)', build_url).group(1)

    # get buildid
    buildid_reponse = requests.get(f"https://www.octopus-code.org/buildbot/api/v2/builders/{builders_id}/builds/{builds_id}")
    buildid = json.loads(buildid_reponse.content)['builds'][0]['buildid']  # buildid = 61409

    # get stepid  from 'https://www.octopus-code.org/buildbot/api/v2/builds/61409/steps/5'
    stepid_response = requests.get(f"https://www.octopus-code.org/buildbot/api/v2/builds/{buildid}/steps/5")
    stepid = json.loads(stepid_response.content)['steps'][0]['stepid'] # stepid = 568186

    # get logid from https://www.octopus-code.org/buildbot/api/v2/steps/568186/logs/config_h
    logid_response = requests.get(f"https://www.octopus-code.org/buildbot/api/v2/steps/{stepid}/logs/config_h")
    logid = json.loads(logid_response.content)['logs'][0]['logid'] # logid = 709753

    # get config.h from 'https://www.octopus-code.org/buildbot/api/v2/logs/709753/contents'
    config_h_response = requests.get(f"https://www.octopus-code.org/buildbot/api/v2/logs/{logid}/contents")
    config_h = json.loads(config_h_response.content)["logchunks"][0]["content"]
    return config_h.replace('o\n','\n').replace('\no','\n')


In [4]:
sample_config_h = get_config_h( "https://www.octopus-code.org/buildbot/#/builders/144/builds/194")

In [5]:
print(sample_config_h)

o/* config.h.  Generated from config.h.in by configure.  */
/* config.h.in.  Generated from configure.ac by autoheader.  */

/* date when configure was launched */
#define BUILD_TIME "Wed Dec  6 10:51:00 CET 2023"

/* C compiler */
#define CC "gcc "

/* The C type of a Fortran integer */
#define CC_FORTRAN_INT int

/* C compiler flags */
#define CFLAGS "-Wall -O2 -march=native -ftest-coverage -fprofile-arcs"

/* C compiler flags (extra) */
#define CFLAGS_EXTRA ""

/* C++ compiler */
#define CXX "g++ "

/* C++ compiler flags */
#define CXXFLAGS "-Wall -O2 -march=native -ftest-coverage -fprofile-arcs"

/* C++ compiler flags (extra) */
#define CXXFLAGS_EXTRA ""

/* Define to 1 if using 'alloca.c'. */
/* #undef C_ALLOCA */

/* compiler supports line-number lines */
#define F90_ACCEPTS_LINE_NUMBERS 1

/* Fortran compiler */
#define FC "gfortran "

/* Fortran compiler flags */
#define FCFLAGS "-Wall -Wno-maybe-uninitialized -Wno-surprising -O2 -march=native -fbacktrace -ftest-coverage -fprof

In [6]:
print(get_config_h("https://www.octopus-code.org/buildbot/#/builders/144/builds/192"))

o/* config.h.  Generated from config.h.in by configure.  */
/* config.h.in.  Generated from configure.ac by autoheader.  */

/* date when configure was launched */
#define BUILD_TIME "Wed Dec  6 09:29:51 CET 2023"

/* C compiler */
#define CC "gcc "

/* The C type of a Fortran integer */
#define CC_FORTRAN_INT int

/* C compiler flags */
#define CFLAGS "-Wall -O2 -march=native -ftest-coverage -fprofile-arcs"

/* C compiler flags (extra) */
#define CFLAGS_EXTRA ""

/* C++ compiler */
#define CXX "g++ "

/* C++ compiler flags */
#define CXXFLAGS "-Wall -O2 -march=native -ftest-coverage -fprofile-arcs"

/* C++ compiler flags (extra) */
#define CXXFLAGS_EXTRA ""

/* Define to 1 if using 'alloca.c'. */
/* #undef C_ALLOCA */

/* compiler supports line-number lines */
#define F90_ACCEPTS_LINE_NUMBERS 1

/* Fortran compiler */
#define FC "gfortran "

/* Fortran compiler flags */
#define FCFLAGS "-Wall -Wno-maybe-uninitialized -Wno-surprising -O2 -march=native -fbacktrace -ftest-coverage -fprof

# Get a diff

In [39]:
import difflib

from rich.markdown import Markdown
from rich import print as rprint
import inspect

In [8]:
f1_content = get_config_h("https://www.octopus-code.org/buildbot/#/builders/144/builds/194")
f2_content = get_config_h("https://www.octopus-code.org/buildbot/#/builders/144/builds/192")

In [31]:
diff = difflib.unified_diff(
                f1_content.split("\n"),
                f2_content.split("\n"),
                "left",
                "right",
            )
diffy = "\n".join(list(diff))

In [32]:
diffy_body =Markdown(
f"""
```diff
{diffy}
```
""",
code_theme="vim",
)

In [33]:
rprint(diffy_body)

                                                                                                                   
 --- left                                                                                                          
                                                                                                                   
 +++ right                                                                                                         
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Wed Dec  6 10:51:00 CET 2023"                                                                
 +#define BUILD_TIME "Wed Dec  6 09:29:51 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "gcc "                                                                                                
 @@ -73,7 +73,7 @@                                                                                                 
                                                                                                                   
  #define FC_INTEGER_SIZE 4                                                                                        
                                                                                                                   
  /* git commit hash */                                                                                            
 -#define GIT_COMMIT "2db69a4030c6cf72ab92cfc37b31671a23b2541f"                                                    
 +#define GIT_COMMIT "01319e380b0e8f85703bf2d9c6652877a52afa53"                                                    
                                                                                                                   
  /* Define to 1 if you have 'alloca', as a function or macro. */                                                  
  #define HAVE_ALLOCA 1                                                                                            
                                                                                                                   

In [44]:
def get_diff(str_old:str, str_new:str,str_old_name="old",str_new_name="new"):
    diff = difflib.unified_diff(
                str_old.split("\n"),
                str_new.split("\n"),
                str_old_name,
                str_new_name
            )
    diffy = "\n".join(list(diff))

    diffy_body = Markdown(
    f"""
```diff
{diffy}
```
    """,
    code_theme="vim",
    )

    return diffy_body

In [46]:
get_diff(f1_content,f2_content)

                                                                                                                   
 --- old                                                                                                           
                                                                                                                   
 +++ new                                                                                                           
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Wed Dec  6 10:51:00 CET 2023"                                                                
 +#define BUILD_TIME "Wed Dec  6 09:29:51 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "gcc "                                                                                                
 @@ -73,7 +73,7 @@                                                                                                 
                                                                                                                   
  #define FC_INTEGER_SIZE 4                                                                                        
                                                                                                                   
  /* git commit hash */                                                                                            
 -#define GIT_COMMIT "2db69a4030c6cf72ab92cfc37b31671a23b2541f"                                                    
 +#define GIT_COMMIT "01319e380b0e8f85703bf2d9c6652877a52afa53"                                                    
                                                                                                                   
  /* Define to 1 if you have 'alloca', as a function or macro. */                                                  
  #define HAVE_ALLOCA 1                                                                                            
                                                                                                                   


# Diff eb(left) vs spack(right)

In [47]:
# Builder map for branch "spack-builder-tollerances-round2" revision "2552abad9f91a5d93a046214bb6520d44507a8f1"
# test suite app https://octopus-code.org/testsuite/commits/1636
# eb pipeline https://gitlab.com/octopus-code/octopus/-/pipelines/1094456375
# spack pipeline https://www.octopus-code.org/buildbot/#/builders/137/builds/17

# builder_map = {builder:[eb_url,spack_url]}
builder_map = {
    "foss2022a_serial_min": [
        "_",
        "https://www.octopus-code.org/buildbot/#/builders/144/builds/155",
    ],
    "foss2022a_serial": [
        "https://www.octopus-code.org/buildbot/#/builders/51/builds/1775",
        "https://www.octopus-code.org/buildbot/#/builders/100/builds/35",
    ],
    "foss2022a_serial_omp": [
        "https://www.octopus-code.org/buildbot/#/builders/31/builds/950",
        "https://www.octopus-code.org/buildbot/#/builders/139/builds/20",
    ],
    "foss2022a_serial_debug": [
        "https://www.octopus-code.org/buildbot/#/builders/37/builds/1756",
        "https://www.octopus-code.org/buildbot/#/builders/151/builds/10",
    ],
    "foss2022a_mpi": [
        "https://www.octopus-code.org/buildbot/#/builders/41/builds/1802",
        "https://www.octopus-code.org/buildbot/#/builders/102/builds/34",
    ],
    "foss2022a_mpi_omp": [
        "https://www.octopus-code.org/buildbot/#/builders/53/builds/970",
        "https://www.octopus-code.org/buildbot/#/builders/138/builds/17",
    ],
    "foss2022a_mpi_debug": [
        "https://www.octopus-code.org/buildbot/#/builders/25/builds/44",
        "https://www.octopus-code.org/buildbot/#/builders/152/builds/11",
    ],
    "foss2022a_cuda_mpi_omp": [
        "https://www.octopus-code.org/buildbot/#/builders/147/builds/20",
        "https://www.octopus-code.org/buildbot/#/builders/159/builds/5",
    ],
}


In [51]:
for builder in builder_map:
    eb_url = builder_map[builder][0]
    spack_url = builder_map[builder][1]
    headding = Markdown(f"""
## {builder=}
### {eb_url=}
### {spack_url=}
""")
    rprint(headding)
    try:
        eb_config_h = get_config_h(eb_url)
        spack_config_h = get_config_h(spack_url)
        diff = get_diff(
            eb_config_h, spack_config_h, f"eb_{builder}", f"spack_{builder}"
        )
        rprint(diff)
    except:
        rprint(Markdown("### failed to compile results"))

builder='foss2022a_serial_min'                                           

                     eb_url='https://www.octopus-code.org/buildbot/#/builders/64/builds/1402'                      

                    spack_url='https://www.octopus-code.org/buildbot/#/builders/144/builds/155'

failed to compile results

builder='foss2022a_serial'                                             

                     eb_url='https://www.octopus-code.org/buildbot/#/builders/51/builds/1775'                      

                    spack_url='https://www.octopus-code.org/buildbot/#/builders/100/builds/35'

                                                                                                                   
 --- eb_foss2022a_serial                                                                                           
                                                                                                                   
 +++ spack_foss2022a_serial                                                                                        
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Tue Dec  5 22:39:14 CET 2023"                                                                
 +#define BUILD_TIME "Mon Dec  4 14:17:04 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "gcc "                                                                                                
 @@ -35,7 +35,7 @@                                                                                                 
                                                                                                                   
  #define FC "gfortran "                                                                                           
                                                                                                                   
  /* Fortran compiler flags */                                                                                     
 -#define FCFLAGS "-Wall -Wno-maybe-uninitialized -Wno-surprising -O2 -march=native -fbacktrace -ftest-coverage    
 -fprofile-arcs"                                                                                                   
 +#define FCFLAGS "-Wall -Wno-maybe-uninitialized -Wno-surprising -O2 -march=native -fbacktrace  -ftest-coverage   
 -fprofile-arcs"                                                                                                   
                                                                                                                   
  /* Fortran compiler flags (extra) */                                                                             
  #define FCFLAGS_EXTRA ""                                                                                         
 @@ -127,7 +127,7 @@                                                                                               
                                                                                                                   
  /* #undef HAVE_CL_CL_H */                                                                                        
                                                                                                                   
  /* This is defined when we link with an external ISF library. */                                                 
 -/* #undef HAVE_COMP_ISF */                                                                                       
 +#define HAVE_COMP_ISF 1                                                                                          
                                                        

builder='foss2022a_serial_omp'                                           

                      eb_url='https://www.octopus-code.org/buildbot/#/builders/31/builds/950'                      

                    spack_url='https://www.octopus-code.org/buildbot/#/builders/139/builds/20'

...

                                                                                                                   
 --- eb_foss2022a_mpi_omp                                                                                          
                                                                                                                   
 +++ spack_foss2022a_mpi_omp                                                                                       
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Tue Dec  5 22:39:22 CET 2023"                                                                
 +#define BUILD_TIME "Mon Dec  4 14:17:09 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "mpicc (gcc)"                                                                                         
 @@ -32,7 +32,7 @@                                                                                                 
                                                                                                                   
  #define F90_ACCEPTS_LINE_NUMBERS 1                                                                               
                                                                                                                   
  /* Fortran compiler */                                                                                           
 -#define FC "mpifort (gfortran)"                                                                                  
 +#define FC "mpif90 (gfortran)"                                                                                   
                                                                                                                   
  /* Fortran compiler flags */                                                                                     
  #define FCFLAGS "-Wall -Wno-maybe-uninitialized -Wno-surprising -O3 -march=native -fbacktrace -funroll-loops     
 -fopenmp"                                                                                                         
 @@ -85,7 +85,7 @@                                                                                                 
                                                                                                                   
  #define HAVE_ALPHASORT 1                                                                                         
                                                                                                                   
  /* Defined if you have the bigdft-atlab library. */                                                              
 -/* #undef HAVE_ATLAB */                                                                                          
 +#define HAVE_ATLAB 1                                                                                             
                                                                                                                   
  /* Defined if you have the BerkeleyGW library. */     

builder='foss2022a_mpi_debug'                                           

                      eb_url='https://www.octopus-code.org/buildbot/#/builders/25/builds/44'                       

                    spack_url='https://www.octopus-code.org/buildbot/#/builders/152/builds/11'

                                                                                                                   
 --- eb_foss2022a_mpi_debug                                                                                        
                                                                                                                   
 +++ spack_foss2022a_mpi_debug                                                                                     
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Tue Dec  5 22:40:00 CET 2023"                                                                
 +#define BUILD_TIME "Mon Dec  4 14:18:00 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "mpicc (gcc)"                                                                                         
 @@ -32,13 +32,13 @@                                                                                               
                                                                                                                   
  #define F90_ACCEPTS_LINE_NUMBERS 1                                                                               
                                                                                                                   
  /* Fortran compiler */                                                                                           
 -#define FC "mpifort (gfortran)"                                                                                  
 +#define FC "mpif90 (gfortran)"                                                                                   
                                                                                                                   
  /* Fortran compiler flags */                                                                                     
  #define FCFLAGS "-g -fno-var-tracking-assignments -Wall -Wno-maybe-uninitialized -Wno-surprising -O2 -march=nati 
 -fbacktrace -fcheck=al"                                                                                           
                                                                                                                   
  /* Fortran compiler flags (extra) */                                                                             
 -#define FCFLAGS_EXTRA "l -fbounds-check -finit-real=snan -ffpe-trap=zero,invalid -ftest-coverage -fprofile-arcs" 
 +#define FCFLAGS_EXTRA "l -fbounds-check -finit-real=snan -finit-logical=false -finit-derived                     
 -ffpe-trap=zero,invalid -ftest-coverage -fprofile-"                                                               
                                                                                                                   
  /* compiler supports command line arguments */                                                                   
  #define FC_COMMAND_LINE_ARGUMENTS 2003                                                                           
 @@ -88,10 +88,10 @@                                    

builder='foss2022a_cuda_mpi_omp'                                          

                      eb_url='https://www.octopus-code.org/buildbot/#/builders/147/builds/20'                      

                     spack_url='https://www.octopus-code.org/buildbot/#/builders/159/builds/5'

                                                                                                                   
 --- eb_foss2022a_cuda_mpi_omp                                                                                     
                                                                                                                   
 +++ spack_foss2022a_cuda_mpi_omp                                                                                  
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Mon Dec  4 15:44:43 CET 2023"                                                                
 +#define BUILD_TIME "Mon Dec  4 14:56:11 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "mpicc (gcc)"                                                                                         
 @@ -11,7 +11,7 @@                                                                                                 
                                                                                                                   
  #define CC_FORTRAN_INT int                                                                                       
                                                                                                                   
  /* C compiler flags */                                                                                           
 -#define CFLAGS "-Wall -O3 -march=native -funroll-loops"                                                          
 +#define CFLAGS "-Wall -O2 -march=native"                                                                         
                                                                                                                   
  /* C compiler flags (extra) */                                                                                   
  #define CFLAGS_EXTRA ""                                                                                          
 @@ -32,13 +32,13 @@                                                                                               
                                                                                                                   
  #define F90_ACCEPTS_LINE_NUMBERS 1                                                                               
                                                                                                                   
  /* Fortran compiler */                                                                                           
 -#define FC "mpifort (gfortran)"                                                                                  
 +#define FC "mpif90 (gfortran)"                                                                                   
                                                                                                                   
  /* Fortran compiler flags */                                                                                     
 -#define FCFLAGS "-Wall -Wno-maybe-uninitialized -Wno-s